Vamos iniciar nosso notebook chamando todas as bibliotecas que usaremos. Nessa etapa, nada precisa ser modificado.

In [ ]:
!pip install arrow bokeh paho.mqtt -q
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import pprint
import numpy as np
import arrow
import requests
import json
import pandas as pd
from threading import Timer

#Url de publicacao dos dados
pub_url = 'https://data.demo.konkerlabs.net/pub/'
#Url da API
base_api = 'https://api.demo.konkerlabs.net'
#Application padrão
application = 'default'

In [ ]:
# Credenciais do usuário da Plataforma
username = ''
password = ''

In [ ]:
device_name = "Cloud"

## Usando a API da Konker para obter os dados e analisa-los localmente
Para iniciar esse trabalho, vamos primeiro conectar na API da Konker. A API usa OAuth2, então primeiro vamos obter as credenciais.

In [ ]:
client = BackendApplicationClient(client_id=username)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='{}/v1/oauth/token'.format(base_api),
                                       client_id=username,
                                       client_secret=password)

In [ ]:
devices = oauth.get("https://api.demo.konkerlabs.net/v1/{}/devices/".format(application)).json()['result']
for dev in devices:
    print(dev)

Vamos procurar pelo dispositivo Cloud na sua lista de dispositivos:

In [ ]:
guid_dev=""
for dev in devices:
    if dev['name'] == device_name:
        guid_dev = dev['guid']

print("O GUID do dispositivo é: "+ guid_dev)

Caso você consiga ver o GUID do dispositivo, significa que está tudo funcionando bem. Caso o GUID não apareça, revise o nome do dispositivo no Notebook e o nome escolhido na plataforma para garantir que eles possuem a mesma grafia.

In [ ]:
dt_start = arrow.utcnow().to('America/Sao_Paulo').floor('day')
dt_start = dt_start.shift(days=-20)
stats = oauth.get("https://api.demo.konkerlabs.net/v1/{}/incomingEvents?q=device:{} timestamp:>{}&sort=oldest&limit=10000".format(application,guid_dev,dt_start.isoformat())).json()['result']
print(stats)

In [ ]:
stats_df = pd.json_normalize(stats).set_index('timestamp')
stats_df

Agora você pode trabalhar com os dados no formato tabular. Lembre-se que cada aplicação do experimento 3 foi enviada em um Canal, facilitando a separação para análise de dados.